In [1]:
import datetime as dt
import pandas as pd
import pickle
import numpy as np
import app_fcns as fcns
from dash import Dash, html, dcc
import plotly.express as px
from plotly.subplots import make_subplots

In [24]:
# Generate parameters

# Get defaults:
# dStart,Bat_capacity,ch_max,Eff,agile_version = import_user_inputs()

# Start date:
dStart='20-Sep-2022 00:00'

# 0:'AGILE-18-02-21', 1:'AGILE-22-07-22', 2:'AGILE-22-08-31', Fixed:'VAR-19-04-12'
agile_version = 2


# Battery parameters:

# Capacity of battery in kWh:
Bat_capacity = 13 

# Max charge/discharge power in kW:
ch_max = 3

# Round-trip efficiency:
Eff = 0.9 


user_params = dict()
user_params['dStart'] = dStart
user_params['Bat_capacity'] = Bat_capacity
user_params['ch_max'] = ch_max
user_params['Eff'] = Eff
user_params['agile_version'] = agile_version

params,batparams,tindex = fcns.get_params(user_params)


In [25]:
# Pull price data from Octopus API

availstr1, availstr2 = fcns.get_tariff_availability(params)
print(availstr1)
print(availstr2)

tariffs = fcns.get_tariffs(params)


AGILE-22-08-31tariff was available in these dates
VAR-19-04-12tariff was available in these dates


In [22]:
import requests
use_tar=3
tar_types = ['AGILE-18-02-21','AGILE-22-07-22','AGILE-22-08-31','VAR-19-04-12']
    
url0 = 'https://api.octopus.energy/v1/'
url1 = 'products/'+tar_types[use_tar]+'/'
url2 = 'electricity-tariffs/E-1R-'+tar_types[use_tar]+'-C/standard-unit-rates/'
url3 = '?period_from='
url4 = params['dStart']
url5 = '&period_to='
url6 = params['dEnd']

url = (url0+url1+url2+url3+url4+url5+url6)

query = """query {
    energyProduct(code:"AGILE-22-08-31") {
    availableFrom
    availableTo
  }
}"""

url = 'https://api.octopus.energy/v1/graphql/'
r = requests.post(url, json={'query': query})
return_dict = r.json()
availFr = return_dict['data']['energyProduct']['availableFrom']
availTo = return_dict['data']['energyProduct']['availableTo']

availability = 1 if params['dStart']>availFr and params['dEnd']<availTo else 0

if availability==1:
    availstr = tar_types[use_tar]+" tariff was available in these dates"
else:
    availstr = tar_types[use_tar]+" tariff was not available in these dates"

print(availstr)

VAR-19-04-12 tariff was not available in these dates


In [23]:
params['dEnd']
print(availTo)

2022-10-06T11:05:00+01:00


In [7]:
# Pull demand data from AWS RDS

dem_df =  fcns.get_demand(params)


In [8]:
# Simulate costs for one set of charging/discharging times

# Start charging at:
ch_start_hour=3

# Start discharging at:
dch_start_hour=14

res_df, cost_df = fcns.run_1_sim(params,batparams,tariffs,dem_df,tindex,ch_start_hour,dch_start_hour)


In [9]:
# Find best charging/disharging times

res_df,cost_df,bestsol,best_dch_start,best_ch_start = fcns.run_best_sim(params,batparams,tariffs,dem_df,tindex)


In [10]:
# Plot results using dash and plotly

from jupyter_plotly_dash import JupyterDash

app = JupyterDash(__name__)

dfa = pd.concat([tariffs['Agile'],tariffs['Fixed']],axis=1)
dfa.columns=['Agile','Fixed']
dfa=dfa.melt(var_name='tariff', value_name='value', ignore_index=False)

df1 = cost_df.cumsum()

subfig = make_subplots(specs=[[{"secondary_y": True}]])

fig = px.line(res_df, x=res_df.index, y=["Baseline_Grid","Grid"],color_discrete_sequence=px.colors.qualitative.Vivid)
figa = px.line(dfa, x=dfa.index, y=["value"],color='tariff',line_dash_sequence=['dot'],width=10)

figa.update_traces(yaxis="y2")

subfig.add_traces(fig.data + figa.data)
subfig.layout.xaxis.title="Time"
subfig.layout.yaxis.title="Power Consumption"
subfig.layout.yaxis2.title="price p/kWh"
subfig.layout.yaxis2.showgrid=False


bars=pd.DataFrame(data=[df1.iloc[-1].values],columns=df1.columns,index=['Total cost']).T


figures = [
            px.line(df1, x=df1.index, y=["Agile","Fixed"], title="Test",color_discrete_sequence=px.colors.qualitative.Vivid),
            px.bar(bars)
    ]

fig1 = make_subplots(rows=1, cols=len(figures)) 
fig1.layout.yaxis.title="Accumulated cost £"

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        fig1.append_trace(figure["data"][trace], row=1, col=i+1)




app.layout = html.Div(children=[
    html.Div([        
        html.H1(children='Grid demand'),

        html.Div(children='''
            With and without battery+Agile.
        '''),

        dcc.Graph(
            id='f1',
            figure=subfig
        ),
    ]),


    html.Div([
        html.Div([
            html.H1(children='Cumulative cost'),

            html.Div(children='''
                With and without battery+Agile.
            '''),

            dcc.Graph(
                id='f2',
                figure=fig1
            ),
        ]),
    ]),    
])

app
